# Como fazer chamadas simples com API? 

Antes de mais nada, vamos escolher o LLM utilizado, seja ele o ChatGPT (gepeto), o Gemini etc. Caso você escolha o gepeto, terá de fazer um **pip install openai**, caso escolha o gemini, terá de fazer **pip install google**. Uma vez escolhido o LLM, cria um ambiente virtual para isolar o ambiente de desenvolvimento do seu código com **python -m venv nome-do-seu-venv** e depois ative-o **.\venv\Scripts\Activate.ps1**. 

In [ ]:
# LLMs
from openai import OpenAI
from google import genai

# Para validação de campos e geração de uma resposta estruturada
from pydantic import BaseModel

# Para a leitura das chaves no arquivo ".env"
from dotenv import load_dotenv, dotenv_values

In [ ]:
# Protegendo as chaves de API e carregando-as em variáveis
config = dotenv_values(".env")
openai_api_key = config["IRAN_OPENAI_KEY"]
gemini_api_key = config["IRAN_GEMINI_KEY"]

In [ ]:
# Alimentando o client com as devidas chaves de api
client_openai = OpenAI(api_key=openai_api_key)
client_gemini = genai.Client(api_key=gemini_api_key)